In [3]:

from jaqs_fxdayu.data import DataView
from jaqs_fxdayu.data.dataservice import LocalDataService
import numpy as np
import pandas as pd
import warnings
from jaqs_fxdayu.util import dp
start = 20160101
end = 20180101
warnings.filterwarnings("ignore")
dataview_folder = './data'
dv=DataView()
ds = LocalDataService(fp=dataview_folder)
factor_list=['volume']
ZZ800_id = ds.query_index_member("000906.SH", start, end)
stock_symbol = list(set(ZZ800_id))
check_factor=','.join(factor_list)
dv_props={'start_date':start,'end_date':end,'symbol':','.join(stock_symbol),'field':check_factor,'freq':1,'prepare_fields':True
}
dv.init_from_config(dv_props,data_api=ds)
dv.prepare_data()

Initialize config success.
Query data...
Query data - query...
NOTE: price adjust method is [post adjust]
当前请求daily...
{'adjust_mode': None, 'fields': 'high,close_adj,low,trade_status,symbol,open_adj,trade_date,vwap_adj,high_adj,low_adj,close,vwap,open'}
下载进度946/981.
当前请求daily...
{'adjust_mode': 'post', 'fields': 'high,low,close,vwap,open,symbol,trade_date'}
下载进度946/981.
    At fields 000748.SZ, 600005.SH
Query data - daily fields prepared.
Query instrument info...
Query adj_factor...
Data has been successfully prepared.


In [6]:
FinancialExpenseRate=dv.add_formula('FinancialExpenseRate_a',"fin_exp/oper_rev",is_quarterly=False)

Variable [fin_exp] is not recognized (it may be wrong),try to fetch from the server...
Query data - query...
    At fields 000748.SZ, 600005.SH
Query data - quarterly fields prepared.
Query data - query...
    At fields 000748.SZ, 600005.SH
Query data - quarterly fields prepared.
Variable [oper_rev] is not recognized (it may be wrong),try to fetch from the server...
Query data - query...
    At fields 000748.SZ, 600005.SH
Query data - quarterly fields prepared.


In [10]:
import scipy.stats as scst
import jaqs.util as jutill
def spearman_factor(factor_u='factor_u', factor_j='factor_j', start=20160101, symbol=stock_symbol):
    factors = dict()
    factors["factor_u"] = dv.get_ts(factor_u).loc[start:][stock_symbol].stack()
    factors["factor_j"] = dv.get_ts(factor_j).loc[start:][stock_symbol].stack()

    factors_df = pd.concat(factors.values(),axis=1)
    factors_df.columns = factors.keys()
    factors_df = jutill.fillinf(factors_df).dropna()

    # 两个因子的spearman相关性
    def get_factors_daily_corr(factors_df,
                               f1="factor_u",
                               f2="factor_j"):
        def _cal_corr(df):
            return scst.spearmanr(df[f1], df[f2])[0]

        result = factors_df.groupby(['trade_date']).apply(_cal_corr)
        result = pd.DataFrame(result)
        result.columns = ['spearmanr']
        return result

    result = get_factors_daily_corr(factors_df,"factor_u","factor_j")
    t_stat, p_value = scst.ttest_1samp(result, 0)
    return result.mean(), p_value # 相关关系均值,相关关系均值的ｔ统计量的p_value

In [8]:
print(spearman_factor(factor_u='FinancialExpenseRate',factor_j='FinancialExpenseRate_a',start=20160101,symbol=stock_symbol))

KeyError: "['002366.SZ' '600643.SH' '000402.SZ' '600115.SH' '600623.SH' '300291.SZ'\n '002011.SZ' '000089.SZ' '600525.SH' '002460.SZ' '002392.SZ' '603883.SH'\n '600126.SH' '000630.SZ' '600594.SH' '002588.SZ' '002477.SZ' '000998.SZ'\n '600160.SH' '600958.SH' '002437.SZ' '600598.SH' '600079.SH' '601600.SH'\n '000951.SZ' '002373.SZ' '600416.SH' '600811.SH' '002415.SZ' '000883.SZ'\n '600362.SH' '600500.SH' '002841.SZ' '300017.SZ' '002140.SZ' '603169.SH'\n '000727.SZ' '300024.SZ' '600468.SH' '600064.SH' '002385.SZ' '600415.SH'\n '002241.SZ' '601225.SH' '300274.SZ' '002027.SZ' '002131.SZ' '000078.SZ'\n '000860.SZ' '002414.SZ' '600563.SH' '002129.SZ' '600036.SH' '000625.SZ'\n '000517.SZ' '600820.SH' '600180.SH' '000898.SZ' '002292.SZ' '600200.SH'\n '600919.SH' '600575.SH' '603806.SH' '002663.SZ' '002261.SZ' '600478.SH'\n '600511.SH' '600881.SH' '600337.SH' '002029.SZ' '600759.SH' '600021.SH'\n '601179.SH' '000559.SZ' '600153.SH' '000738.SZ' '000423.SZ' '603766.SH'\n '002216.SZ' '002194.SZ' '0